# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import logging
import time

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# choose a name for experiment
experiment_name = 'capstoneautoml'

experiment=Experiment(ws, experiment_name)

In [ ]:
amlcompute_cluster_name="runautoml"

try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D3_V2",max_nodes=4)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

In [ ]:
auto_ds_path="https://raw.githubusercontent.com/kgpyi/Machine-Learning/master/Udacity/Machine%20Learning%20Engineer%20with%20Microsoft%20Azure%20Nanodegree%20Program/Capstone%20Project/heart_failure_clinical_records_dataset.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)

In [ ]:
df = ds.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":8,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":20,
    "max_concurrent_iterations":15,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target='runautoml',
    training_data=ds,
    label_column_name='DEATH_EVENT',
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

In [ ]:
print(best_remote_run)
best_remote_run

In [ ]:
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model.register(workspace = ws,
                        model_name = "Heart_Failure_Prediction",
                        model_path = "./automl_model.pkl",
                        description = "AutoML Model which was best according to accuracy",)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#defining an inference configuration
service_name = 'heart-failure-automl'

#setting up a curated environment
from azureml.automl.core.shared import constants
best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, 'AzureML-AutoML.yml')
env = Environment.get(ws, name="AzureML-AutoML")

#combining scoring script and environment in inference configuration
inference_config = InferenceConfig(entry_script='scoring.py', environment=env)

#deploy the model as a webservice
from azureml.core.webservice import LocalWebservice, Webservice

#set deployment configuration
local_deployment_config = LocalWebservice.deploy_configuration(port=8000)
service = Model.deploy(ws, "heart-failure-automl", [model], inference_config, local_deployment_config)

print(service.state)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True) #service.update

service = Model.deploy(workspace=ws,
                        name='heart-failure-automl',
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.swagger_uri)
print(service.scoring_uri)

In [ ]:
primary, secondary= service.get_keys()
print(primary)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               "age":45,
               "anaemia":1,
               "creatinine_phosphokinase":7861,
               "diabetes":1,
               "ejection_fraction":49,
               "high_blood_pressure":0,
               "platelets":276000,
               "serum_creatinine":1,
               "serum_sodium":140,
               "sex":0,
               "smoking":1,
               "time":16
               },
            {
               "age":51,
               "anaemia":0,
               "creatinine_phosphokinase":78,
               "diabetes":0,
               "ejection_fraction":50,
               "high_blood_pressure":0,
               "platelets":406000,
               "serum_creatinine":0.7
               "serum_sodium":140,
               "sex":1,
               "smoking":0,
               "time":79
                }
        ]
        }

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primary}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:

#logs of the webservice
logs=service.get_logs()
print(logs)

In [ ]:

#delete the deployed webservice
service.delete()